In [4]:
import json
for i in [0]:
    data = []
    with open("/Users/philip/Documents/PhD/data/ArenaData/arena_fits/2015-07-05.json") as f:
        data = f.readlines()

    json_lines = []
    
    for line in data:
        jsline = json.loads(line)
        json_lines.append(jsline)

In [28]:
import pandas as pd

frame = pd.DataFrame.from_dict(json_lines)

In [29]:
# make dataframe of dicts nested in 'value' column
newframe = pd.DataFrame(list(frame['value']))

# make dataframe of dicts nested in 'trackeeHistory' column
newframe2 = pd.DataFrame(list(newframe['trackeeHistory']))

In [30]:
# make dataframe with a 'coordinates' column
newframe3 = pd.DataFrame(list(newframe['averagecoordinate']))
newframe4 = pd.DataFrame(list(newframe3['error']))

In [31]:
# make dataframe with only the x-value of coordinates as element
# could also be Series
from math import sqrt

x_error = []
y_error = []
for i in range(len(newframe4)):
    x_error.append(newframe4.coordinates[i][0])
    y_error.append(newframe4.coordinates[i][1])
x_err = pd.DataFrame(x_error)
y_err = pd.DataFrame(y_error)

In [32]:
# delete columns containing dicts and join dataframes
del newframe['trackeeHistory']
newframe3 = newframe.join(newframe2)
del frame['value']
bigframe = frame.join(newframe3)
bigframe['x_error'] = x_err
bigframe['y_error'] = y_err

In [33]:
# get most frequent mac address
macs = bigframe['sourceMac'].value_counts()
macs[:5]

2d1ff804-c499-4163-b90f-003df1e4ec65    78664
85a84e08-c967-41d0-b56b-522574796776    74356
2c019c2d-7e9e-42b1-b3bd-1797c918d4a7    16995
e1deda99-163f-4b68-9ade-e1f05d070bf2     8370
6cab229a-e15d-48b7-ab1e-5ddca3d7e283     7712
Name: sourceMac, dtype: int64

In [ ]:
from math import floor, ceil
import numpy as np

width = 80; height = 60

bins = np.zeros((height,width))

for i in range(100): # len(macs)
    filtered_frame = bigframe[bigframe['sourceMac'] == macs.index[i]]
    # filters
    probChi2_filtered = filtered_frame[filtered_frame['probChi2'] > 0.3]
    error_filtered = probChi2_filtered[probChi2_filtered['x_error'] < 5] # was 5
    error_filtered = probChi2_filtered[probChi2_filtered['y_error'] < 5]
    Chi2dof_filtered = error_filtered[error_filtered['chi2PerDof'] < 1.2] # was 1.2
    # get list of locations
    avgframe = pd.DataFrame(list(Chi2dof_filtered['averagecoordinate']))
    coorframe = pd.DataFrame(list(avgframe['avg']))
    locations = coorframe['coordinates']
    
    positions1 = pd.DataFrame(locations)
    positions = pd.DataFrame(list(positions1['coordinates']))

    f = lambda x: x/3

    # binned positions, still fractional numbers
    binpos = positions.apply(f)

    for j in range(len(binpos) - 1):
        if floor(binpos[0][j]) + 20 >= 0 and floor(binpos[0][j]) + 20 < width and \
            floor(binpos[1][j]) + 15 >= 0 and floor(binpos[1][j]) + 15 < height:
            bins[height/2 + floor(binpos[1][j])][width/2 + floor(binpos[0][j])] += 1/len(positions)
    

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:30: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

fig = plt.figure()

X = np.arange(0,len(bins[1]),1)
Y = np.arange(0,len(bins),1)
Xs, Ys = np.meshgrid(X, Y)

Z = bins

ax = Axes3D(fig)
ax.plot_surface(Xs, Ys, Z, rstride=2, cstride=1, cmap='hsv')#, cmap='hot')
plt.show()

In [107]:
im = plt.imread('/Users/philip/Documents/PhD/data-analysis/escience/arena_sensation.png')

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
# convenience method:
fig, ax = plt.subplots()

#ax.set_xlim((0,1853)) # 1853
#ax.set_ylim((0,1369)) # 1369

ax.imshow(im)
ax.imshow(bins, aspect='auto', cmap='hsv', alpha=1)

#ax.plot(range(1200))

plt.show()

In [109]:
len(Y)

30